# INFINITE POKEMON PIXEL-ART COLOR-BY-NUMBER GENERATOR

This generates color-by-number drawings of imaginary Pokemons. The drawing is different each time and generated by an AI system.

If you are new to this, **the easiest way** is to do the following:

*   Click "Runtime" and then "Run all" or alternatively press Ctrl+F9
*   Wait for the drawing to generate (may take a few minutes)
*   Scroll to the bottom, right-click on the worksheet and select "Save image as" to save the image to your device.
*   Open the saved file and print it for your kids.
*   Have fun



## Credits

Pokemon illustrations are generated using the ruDALL-E model fine-tuned by [Max Woolf](https://minimaxir.com):

[Model on Hugging Face](https://huggingface.co/minimaxir/ai-generated-pokemon-rudalle)

[GitHub Repo](https://github.com/minimaxir/ai-generated-pokemon-rudalle)

### License

MIT License

Copyright (c) 2022 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

---

Original ruDALL-E notebook code adapted under the Apache license located [here](https://github.com/sberbank-ai/ru-dalle/blob/master/LICENSE.txt).

## Generate worksheet

In [ ]:
#@title Setting everything up (may take a few minutes)
!git clone https://github.com/RobertoFont/Blog.git Blog > /dev/null

!pip install rudalle==0.4.0 > /dev/null

from rudalle.pipelines import generate_images, show
from rudalle import get_rudalle_model, get_tokenizer, get_vae
from rudalle.utils import seed_everything
from huggingface_hub import hf_hub_url, cached_download
import os
import torch
import shutil
from IPython.display import clear_output

import sys
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

sys.path.append("Blog/pixel-art-color-by-number-generator")

from utils.image import crop_central, limit_colors, pixelate_image
from utils.sheet import fig2im, get_text_for_cell, generate_color_by_number_sheet

model_filename = "pytorch_model.bin"
device = "cuda"

config_file_url = hf_hub_url(repo_id="minimaxir/ai-generated-pokemon-rudalle", filename=model_filename)
cached_download(config_file_url, cache_dir='/content', force_filename=model_filename)
model = get_rudalle_model('Malevich', pretrained=False, fp16=True, device=device)
model.load_state_dict(torch.load(model_filename, map_location='cpu')) 
vae = get_vae().to(device)
tokenizer = get_tokenizer()

In [ ]:
# @title Worksheet configuration
# @markdown Use the slider to set the desired number of colors.
 
# @markdown `CELLS` can be: `'numbers'` (Each cell displays the number of the corresponding color), `'additions'` (Each cell displays an addition that must be solved to obtain the corresponding color) or `'substractions'` (Each cell displays a subtraction that must be solved to obtain the corresponding color)
NUM_COLORS = 4 #@param {type:"slider", min:2, max:8, step:1}
CELLS = "numbers" #@param ["numbers", "additions", "substractions"]

if CELLS == "numbers":
  MODE = "n"
elif CELLS == "additions":
  MODE = "a"
elif CELLS == "substractions":
  MODE = "s"


In [ ]:
#@title Generate image (may take a few minutes)

# In theory you could get more specific Pokemon by
# specifying the type (see the repo linked in the intro),
# but it practice it does not influence generation much.
text = 'Покемон'

top_k = 128
top_p = 0.95

pil_images, _scores = generate_images(text, tokenizer, model, vae, top_k=top_k, images_num=1, top_p=top_p)

# Pixelize image
im = pil_images[0]
im = crop_central(im)
im = pixelate_image(im)
im, colors = limit_colors(im, NUM_COLORS)

# Preview result
image_h = im.resize((512,512), Image.NEAREST)
display(image_h)


In [ ]:
#@title Generate worksheet (may take a few minutes -again-)

print_ready = generate_color_by_number_sheet(im, colors, mode=MODE)

display(print_ready)


Right-click and "Save image as" to save image to your local drive